In [10]:
from qdrant_client import QdrantClient, models

In [11]:
client = QdrantClient(url="http://localhost:6333")

In [12]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=4, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2, 3, 4, 5, 6],
        payloads=[
            {"color": "red", "city": "London", "age": 24},
            {"color": "green", "city": "Moscow", "age": 34},
            {"color": "blue", "city": "Moscow", "age": 13},
            {"color": "green", "city": "London", "age": 56},
            {"color": "red", "city": "Berlin", "age": 88},
            {"color": "blue", "city": "London", "age": 8},
        ],
        vectors=[
            [0.1, 0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2, 0.2],
            [0.3, 0.3, 0.3, 0.3],
            [0.4, 0.4, 0.4, 0.4],
            [0.5, 0.5, 0.5, 0.5],
            [0.6, 0.6, 0.6, 0.6],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
client.search(
    collection_name="collection0",
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(
                    value="London",
                ),
            )
        ]
    ),
    search_params=models.SearchParams(
        # Params relevant to HNSW index Size of the beam in a beam-search. Larger the value - more accurate the result, more time required for search.
        hnsw_ef=128, 
        # Search without approximation. If set to true, search may run long but with exact results.
        exact=False
        ),
    query_vector=[0.2, 0.1, 0.9, 0.7],
    # Max number of result to return
    limit=3,  # 默认limit=10
    # If `True` - attach all payload
    # If `False` - do not attach any payload
    # If List of string - include only specified fields
    # If `PayloadSelector` - use explicit rules
    with_payload=True,
    # If `True` - Attach stored vector to the search result.
    # If `False` - Do not attach vector.
    # If List of string - Attach only specified vectors.
    with_vectors=True
)

[ScoredPoint(id=6, version=0, score=1.1400001, payload={'age': 8, 'city': 'London', 'color': 'blue'}, vector=[0.6, 0.6, 0.6, 0.6], shard_key=None, order_value=None),
 ScoredPoint(id=4, version=0, score=0.76, payload={'age': 56, 'city': 'London', 'color': 'green'}, vector=[0.4, 0.4, 0.4, 0.4], shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=0.19, payload={'age': 24, 'city': 'London', 'color': 'red'}, vector=[0.1, 0.1, 0.1, 0.1], shard_key=None, order_value=None)]

In [14]:
client.search(
    collection_name="collection0",
    query_vector=[0.2, 0.1, 0.9, 0.7],
    # It is possible to target nested fields using a dot notation:
    #   * payload.nested_field - for a nested field
    #   * payload.nested_array[].sub_field - for projecting nested fields within an array
    with_payload=["city", "color"],
    limit=3
)

[ScoredPoint(id=6, version=0, score=1.1400001, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=5, version=0, score=0.95000005, payload={'city': 'Berlin', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=4, version=0, score=0.76, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None)]

In [15]:
client.search(
    collection_name="collection0",
    query_vector=[0.2, 0.1, 0.9, 0.7],
    with_payload=models.PayloadSelectorExclude(
        exclude=["city"],
    ),
    limit=3
)

[ScoredPoint(id=6, version=0, score=1.1400001, payload={'age': 8, 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=5, version=0, score=0.95000005, payload={'age': 88, 'color': 'red'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=4, version=0, score=0.76, payload={'age': 56, 'color': 'green'}, vector=None, shard_key=None, order_value=None)]